In [1]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os


In [2]:
desired_dataset = 12

cleansed_data = {12: "12dtecleansed.csv"}


DATA_PATH = "/kaggle/input/spx-20162021"
SAVE_PATH = "/kaggle/working/"
filenames = os.listdir(DATA_PATH)

ten_filenames = filenames[0:10]

print(ten_filenames)

for filename in ten_filenames:
    temp = pd.read_csv(f"{DATA_PATH}/{filename}")
    print(temp)

['2016-09-07.csv', '2018-02-01.csv', '2019-06-11.csv', '2020-07-10.csv', '2017-11-20.csv', '2018-12-27.csv', '2017-04-05.csv', '2018-03-02.csv', '2021-06-11.csv', '2016-08-23.csv']
      Unnamed: 0 ticker   tradeDate   expirDate  dte  strike   stockPrice  \
0              0    SPX  2016-09-07  2016-09-07    1    1725  2185.300000   
1              1    SPX  2016-09-07  2016-09-07    1    1750  2185.300000   
2              2    SPX  2016-09-07  2016-09-07    1    1775  2185.300000   
3              3    SPX  2016-09-07  2016-09-07    1    1800  2185.300000   
4              4    SPX  2016-09-07  2016-09-07    1    1825  2185.300000   
...          ...    ...         ...         ...  ...     ...          ...   
3708        3708    SPX  2016-09-07  2018-12-21  836    2750  2086.936601   
3709        3709    SPX  2016-09-07  2018-12-21  836    2800  2086.936601   
3710        3710    SPX  2016-09-07  2018-12-21  836    2900  2086.936601   
3711        3711    SPX  2016-09-07  2018-12-21  

In [3]:
desired_dataset = 12

cleansed_data = {12: "12dtecleansed.csv"}


DATA_PATH = "/kaggle/input/spx-20162021"
SAVE_PATH = "/kaggle/working/"

path = r'E:\Data Science\orats-pyclient\data'
all_files = glob.glob(DATA_PATH + "/*.csv")
df_list = []
for f in all_files:
    print(f)
    temp_df = pd.read_csv(f, index_col=None, header=0)
    df_list.append(temp_df.loc[temp_df['dte']<15])
    
df = pd.concat(df_list, axis=0, ignore_index=True)

#Change string columns to datetime64
df['tradeDate']=pd.to_datetime(df['tradeDate'])
df['expirDate']=pd.to_datetime(df['expirDate'])
df['quoteDate']=pd.to_datetime(df['quoteDate'])
df['updatedAt']=pd.to_datetime(df['updatedAt'])

#Set up trade and exp date series
ser_expDate = pd.Series(df['expirDate'].unique()).sort_values()
ser_tradeDate = pd.Series(df['tradeDate'].unique()).sort_values()

df.to_csv('cleaned.csv')


/kaggle/input/spx-20162021/2016-09-07.csv
/kaggle/input/spx-20162021/2018-02-01.csv
/kaggle/input/spx-20162021/2019-06-11.csv
/kaggle/input/spx-20162021/2020-07-10.csv
/kaggle/input/spx-20162021/2017-11-20.csv
/kaggle/input/spx-20162021/2018-12-27.csv
/kaggle/input/spx-20162021/2017-04-05.csv
/kaggle/input/spx-20162021/2018-03-02.csv
/kaggle/input/spx-20162021/2021-06-11.csv
/kaggle/input/spx-20162021/2016-08-23.csv
/kaggle/input/spx-20162021/2018-05-31.csv
/kaggle/input/spx-20162021/2020-10-15.csv
/kaggle/input/spx-20162021/2020-10-21.csv
/kaggle/input/spx-20162021/2018-12-10.csv
/kaggle/input/spx-20162021/2016-09-12.csv
/kaggle/input/spx-20162021/2018-11-06.csv
/kaggle/input/spx-20162021/2019-07-02.csv
/kaggle/input/spx-20162021/2018-01-12.csv
/kaggle/input/spx-20162021/2016-02-23.csv
/kaggle/input/spx-20162021/2021-03-23.csv
/kaggle/input/spx-20162021/2020-08-19.csv
/kaggle/input/spx-20162021/2019-09-24.csv
/kaggle/input/spx-20162021/2018-04-19.csv
/kaggle/input/spx-20162021/2017-07